# Libraries

In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import json
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import networkx as nx
from collections import defaultdict, Counter, deque
import matplotlib.pyplot as plt
import time

# Inputs

In [ ]:
EN_PATH = '../Data/Input/conceptnet_en_full.csv'

# Purpose
***

Allow an agent to interact with the full relation universe and create a full knowledge graph.

### Phase 1. 
***
Load the english triples

In [ ]:
english_triples = pd.read_csv(EN_PATH)
english_triples.head()

#### Phase 1.A 
***
Preprocess and prepare the concept data

In [ ]:
english_triples['weight'].iloc[0]

In [ ]:
def clean_column(df, col):
    col_string = df[col].astype(str)
    col_string = col_string.str.split('/')
    col_string = col_string.str[-1]
    df[f'{col}_cleaned'] = col_string
    return df

def extract_weight(df):
    df['weight_cleaned'] = df['weight'].apply(lambda x: json.loads(x)['weight'])
    return df

def preprocess_data(df):
    df_copy = df.copy()
    for col in ['relation', 'start', 'end']:
        df_copy = clean_column(df_copy, col)
    df_copy = extract_weight(df_copy)
    return df_copy

cleaned_cols = ['relation', 'start', 'end', 'weight']
cleaned_cols = [f'{col}_cleaned' for col in cleaned_cols]

reprocess_triples = True

if reprocess_triples:
    cleaned_english_triples = preprocess_data(english_triples)[cleaned_cols].drop_duplicates()
    cleaned_english_triples[cleaned_cols].head()
    cleaned_english_triples = cleaned_english_triples.rename(columns={
        'relation_cleaned': 'relation_type',
        'start_cleaned': 'start_concept',
        'end_cleaned': 'end_concept',
        'weight_cleaned': 'edge_weight'
    })
    # Save as a parquet file
    cleaned_english_triples.to_parquet(
        os.path.join(os.path.dirname(EN_PATH), 'conceptnet_en_full_cleaned.parquet.gzip'),
        index=False,
        compression='gzip'
    )
else:
    # Load the cleaned data
    cleaned_english_triples = pd.read_parquet(
        os.path.join(os.path.dirname(EN_PATH), 'conceptnet_en_full_cleaned.parquet.gzip')
    )
cleaned_english_triples.head()

# Phase 2. Seed the agent with random english relations
***

In [ ]:
def create_stratified_weighted_sample(df, sample_size=5000, min_weight_threshold=0.5, verbose=True):
    """
    Create a stratified sample maintaining relation distribution while prioritizing higher weights
    
    Parameters:
    - df: DataFrame with columns ['start_concept', 'end_concept', 'relation_type', 'edge_weight']
    - sample_size: Target number of triples in sample
    - min_weight_threshold: Minimum weight to consider (filters low-quality relations)
    - verbose: Print detailed progress
    
    Returns:
    - DataFrame: Stratified sample
    """
    
    # df_copy = df.copy()
    # df_copy = df_copy.rename(columns={
    #     'relation_cleaned': 'relation_type',
    #     'start_cleaned': 'start_concept',
    #     'end_cleaned': 'end_concept',
    #     'weight_cleaned': 'edge_weight'
    # })
    
    print(f"🎯 Creating stratified weighted sample of {sample_size:,} triples...")
    
    # Step 1: Filter by minimum weight threshold
    if verbose:
        print(f"   Filtering triples with weight >= {min_weight_threshold}")
    
    initial_count = len(df)
    filtered_df = df[df['edge_weight'] >= min_weight_threshold].copy()
    filtered_count = len(filtered_df)
    
    if verbose:
        print(f"   Kept {filtered_count:,} of {initial_count:,} triples ({filtered_count/initial_count*100:.1f}%)")
    
    # Step 2: Calculate current relation distribution
    relation_counts = filtered_df['relation_type'].value_counts()
    relation_proportions = relation_counts / len(filtered_df)
    
    if verbose:
        print(f"\n📊 Original relation distribution (top 10):")
        for relation, prop in relation_proportions.head(10).items():
            count = relation_counts[relation]
            print(f"   {relation}: {count:,} ({prop*100:.1f}%)")
    
    # Step 3: Sort by weight within each relation (highest first)
    if verbose:
        print(f"\n⚖️  Sorting by weight within each relation...")
    
    filtered_df = filtered_df.sort_values(['relation_type', 'edge_weight'], 
                                        ascending=[True, False])
    
    # Step 4: Calculate target samples per relation
    target_samples_per_relation = {}
    for relation in relation_proportions.index:
        target_count = int(sample_size * relation_proportions[relation])
        # Ensure at least 1 sample for each relation if possible
        target_count = max(1, target_count)
        target_samples_per_relation[relation] = target_count
    
    if verbose:
        print(f"\n🎯 Target samples per relation:")
        total_targeted = sum(target_samples_per_relation.values())
        for relation, target in sorted(target_samples_per_relation.items(), 
                                     key=lambda x: x[1], reverse=True)[:10]:
            print(f"   {relation}: {target:,}")
        print(f"   Total targeted: {total_targeted:,}")
    
    # Step 5: Sample from each relation group
    sampled_dfs = []
    actual_samples = {}
    
    if verbose:
        print(f"\n🔄 Sampling from each relation group...")
    
    for relation, target_count in tqdm(target_samples_per_relation.items(), 
                                     desc="Sampling relations"):
        relation_data = filtered_df[filtered_df['relation_type'] == relation]
        
        # Take top weighted samples up to target count
        actual_count = min(target_count, len(relation_data))
        sampled_data = relation_data.head(actual_count)
        
        sampled_dfs.append(sampled_data)
        actual_samples[relation] = actual_count
    
    # Step 6: Combine all samples
    stratified_sample = pd.concat(sampled_dfs, ignore_index=True)
    
    # Step 7: If we're short, fill with highest-weight remaining samples
    current_size = len(stratified_sample)
    if current_size < sample_size:
        shortage = sample_size - current_size
        if verbose:
            print(f"   Short by {shortage:,} samples, filling with highest-weight remaining...")
        
        # Get samples not already included
        used_indices = set(stratified_sample.index) if hasattr(stratified_sample, 'index') else set()
        remaining_df = filtered_df[~filtered_df.index.isin(used_indices)]
        
        if len(remaining_df) > 0:
            # Sort by weight and take top samples
            top_remaining = remaining_df.nlargest(shortage, 'edge_weight')
            stratified_sample = pd.concat([stratified_sample, top_remaining], ignore_index=True)
    
    # Step 8: Final shuffle to mix relations
    stratified_sample = stratified_sample.sample(frac=1.0, random_state=42).reset_index(drop=True)
    
    # Step 9: Validation and statistics
    final_size = len(stratified_sample)
    final_relation_counts = stratified_sample['relation_type'].value_counts()
    final_relation_proportions = final_relation_counts / final_size
    
    print(f"\n✅ Stratified sample created!")
    print(f"   Final size: {final_size:,} triples")
    print(f"   Weight range: {stratified_sample['edge_weight'].min():.3f} - {stratified_sample['edge_weight'].max():.3f}")
    print(f"   Mean weight: {stratified_sample['edge_weight'].mean():.3f}")
    
    if verbose:
        print(f"\n📊 Final relation distribution (top 10):")
        for relation, prop in final_relation_proportions.head(10).items():
            count = final_relation_counts[relation]
            original_prop = relation_proportions.get(relation, 0)
            print(f"   {relation}: {count:,} ({prop*100:.1f}% vs {original_prop*100:.1f}% orig)")
    
    return stratified_sample

#### Phase 2A. 
***
Created stratified sampling for agent seeded nodes

In [ ]:
resample_data = True

if resample_data:
    # Apply the stratified sampling to your cleaned data
    print("🎲 Creating stratified weighted sample for agent initialization...")

    stratified_seed_data = create_stratified_weighted_sample(
        df=cleaned_english_triples,
        sample_size=5000,
        min_weight_threshold=0.5,  # Only include relations with decent confidence
        verbose=True
    )

    # Show sample characteristics
    print(f"\n🔍 Sample characteristics:")
    print(f"Weight distribution:")
    print(stratified_seed_data['edge_weight'].describe())

    print(f"\nTop concept pairs by weight:")
    top_weighted = stratified_seed_data.nlargest(5, 'edge_weight')
    for _, row in top_weighted.iterrows():
        print(f"   {row['start_concept']} --{row['relation_type']}--> {row['end_concept']} (weight: {row['edge_weight']:.3f})")
    # Save the stratified sample
    stratified_output_path = os.path.join(os.path.dirname(EN_PATH), 'conceptnet_en_stratified_seed_5k.parquet.gzip')
    stratified_seed_data.to_parquet(stratified_output_path, index=False, compression='gzip')
    print(f"\n💾 Stratified seed data saved to: {stratified_output_path}")
else:
    # Load the stratified sample
    stratified_seed_data = pd.read_parquet(
        os.path.join(os.path.dirname(EN_PATH), 'conceptnet_en_full_stratified.parquet.gzip')
    )
    print(f"Loaded {len(stratified_seed_data):,} triples from stratified sample.")

# Phase 3 
***
Agent Initialization

In [ ]:
class KnowledgeGraphAgent:
    """
    MVP Knowledge Graph Agent with self-validation and extensible structure
    """
    
    def __init__(self, validate_on_add=True, verbose=True):
        self.graph = nx.MultiDiGraph()  # Allows multiple edges between same nodes
        self.validate_on_add = validate_on_add
        self.verbose = verbose
        self.validation_stats = {
            'total_attempted': 0,
            'successful_adds': 0,
            'duplicates_rejected': 0,
            'contradictions_found': 0,
            'validation_errors': 0
        }
        
        # Contradiction rules - relations that shouldn't coexist
        self.contradiction_rules = {
            'Antonym': ['Synonym', 'RelatedTo'],
            'Synonym': ['Antonym'],
            'Causes': ['Prevents'],
            'Prevents': ['Causes']
        }
        
        print("🧠 Knowledge Graph Agent initialized!")
        print(f"   Validation: {'ON' if validate_on_add else 'OFF'}")
        print(f"   Verbose mode: {'ON' if verbose else 'OFF'}")
    
    def clean_conceptnet_data(self, df):
        """
        Properly clean ConceptNet data - fixes the string splitting issue
        """
        print("🧹 Cleaning ConceptNet data...")
        
        def extract_concept(concept_string):
            """Extract clean concept from ConceptNet URI format"""
            if pd.isna(concept_string):
                return None
            
            # ConceptNet format: /c/en/concept_name/part_of_speech
            # We want the concept_name part
            parts = str(concept_string).split('/')
            if len(parts) >= 4 and parts[1] == 'c' and parts[2] == 'en':
                concept = parts[3]
                # Handle underscores and clean up
                concept = concept.replace('_', ' ')
                return concept
            return concept_string
        
        def extract_relation(relation_string):
            """Extract relation type from ConceptNet URI"""
            if pd.isna(relation_string):
                return None
            parts = str(relation_string).split('/')
            if len(parts) >= 3 and parts[1] == 'r':
                return parts[2]
            return relation_string
        
        def extract_weight(weight_string):
            """Extract numerical weight from JSON string"""
            try:
                weight_data = json.loads(weight_string)
                return float(weight_data.get('weight', 1.0))
            except:
                return 1.0
        
        # Apply cleaning functions
        cleaned_df = df.copy()
        
        print("   Extracting concepts and relations...")
        cleaned_df['start_concept'] = df['start'].apply(extract_concept)
        cleaned_df['end_concept'] = df['end'].apply(extract_concept)
        cleaned_df['relation_type'] = df['relation'].apply(extract_relation)
        cleaned_df['edge_weight'] = df['weight'].apply(extract_weight)
        
        # Filter out invalid entries
        initial_count = len(cleaned_df)
        cleaned_df = cleaned_df.dropna(subset=['start_concept', 'end_concept', 'relation_type'])
        final_count = len(cleaned_df)
        
        print(f"   Filtered {initial_count - final_count:,} invalid entries")
        print(f"   Clean dataset: {final_count:,} triples")
        
        return cleaned_df[['start_concept', 'end_concept', 'relation_type', 'edge_weight']]
    
    def validate_triple(self, start, relation, end, weight=1.0):
        """
        Validate a triple before adding to graph
        Returns: (is_valid, reason)
        """
        # Check for duplicates
        if self.graph.has_edge(start, end):
            existing_edges = self.graph[start][end]
            for edge_data in existing_edges.values():
                if edge_data.get('relation') == relation:
                    return False, f"Duplicate: {start} --{relation}--> {end}"
        
        # Check for contradictions
        if relation in self.contradiction_rules:
            contradictory_relations = self.contradiction_rules[relation]
            
            if self.graph.has_edge(start, end):
                for edge_data in self.graph[start][end].values():
                    if edge_data.get('relation') in contradictory_relations:
                        return False, f"Contradiction: {start} already has {edge_data.get('relation')} with {end}"
        
        # Passed all validation checks
        return True, "Valid"
    
    def add_triple(self, start, relation, end, weight=1.0, force=False):
        """
        Add a validated triple to the knowledge graph
        """
        self.validation_stats['total_attempted'] += 1
        
        if not force and self.validate_on_add:
            is_valid, reason = self.validate_triple(start, relation, end, weight)
            
            if not is_valid:
                if "Duplicate" in reason:
                    self.validation_stats['duplicates_rejected'] += 1
                elif "Contradiction" in reason:
                    self.validation_stats['contradictions_found'] += 1
                else:
                    self.validation_stats['validation_errors'] += 1
                
                if self.verbose:
                    print(f"❌ Rejected: {reason}")
                return False
        
        # Add the triple to graph
        self.graph.add_edge(start, end, relation=relation, weight=weight)
        self.validation_stats['successful_adds'] += 1
        
        if self.verbose and self.validation_stats['total_attempted'] % 1000 == 0:
            print(f"✅ Added {self.validation_stats['successful_adds']:,} triples so far...")
        
        return True
    
    def bulk_load_triples(self, df, max_triples=None):
        """
        Efficiently load multiple triples with progress tracking
        """
        print(f"📊 Loading triples into knowledge graph...")
        
        if max_triples:
            df = df.head(max_triples)
            print(f"   Limited to first {max_triples:,} triples")
        
        total_rows = len(df)
        print(f"   Processing {total_rows:,} triples...")
        
        # Use tqdm for progress bar
        for idx, row in tqdm(df.iterrows(), total=total_rows, desc="Loading triples"):
            self.add_triple(
                start=str(row['start_concept']),
                relation=str(row['relation_type']),
                end=str(row['end_concept']),
                weight=float(row['edge_weight'])
            )
        
        self.print_stats()
    
    def print_stats(self):
        """Print comprehensive statistics about the knowledge graph"""
        print("\n📈 Knowledge Graph Statistics:")
        print("="*50)
        print(f"🔢 Total nodes: {self.graph.number_of_nodes():,}")
        print(f"🔗 Total edges: {self.graph.number_of_edges():,}")
        print(f"📊 Average degree: {np.mean([d for n, d in self.graph.degree()]):,.2f}")
        
        print(f"\n🔍 Validation Results:")
        print(f"   Attempted additions: {self.validation_stats['total_attempted']:,}")
        print(f"   ✅ Successful: {self.validation_stats['successful_adds']:,}")
        print(f"   🔄 Duplicates rejected: {self.validation_stats['duplicates_rejected']:,}")
        print(f"   ⚡ Contradictions found: {self.validation_stats['contradictions_found']:,}")
        print(f"   ❌ Other errors: {self.validation_stats['validation_errors']:,}")
        
        success_rate = (self.validation_stats['successful_adds'] / max(self.validation_stats['total_attempted'], 1)) * 100
        print(f"   📊 Success rate: {success_rate:.2f}%")
        
        # Relation type distribution
        relation_counts = Counter()
        for _, _, data in self.graph.edges(data=True):
            relation_counts[data.get('relation', 'Unknown')] += 1
        
        print(f"\n🏷️  Top 10 Relation Types:")
        for relation, count in relation_counts.most_common(10):
            print(f"   {relation}: {count:,}")
    
    def query_concept(self, concept, max_results=10):
        """
        Query all relations for a given concept
        """
        if concept not in self.graph:
            print(f"❓ Concept '{concept}' not found in graph")
            return []
        
        print(f"🔍 Relations for '{concept}':")
        
        results = []
        
        # Outgoing relations
        for neighbor in list(self.graph.neighbors(concept))[:max_results//2]:
            edge_data = self.graph[concept][neighbor]
            for edge in edge_data.values():
                relation = edge.get('relation', 'Unknown')
                weight = edge.get('weight', 1.0)
                results.append((concept, relation, neighbor, weight, 'outgoing'))
                print(f"   {concept} --{relation}--> {neighbor} (weight: {weight:.2f})")
        
        # Incoming relations
        for predecessor in list(self.graph.predecessors(concept))[:max_results//2]:
            edge_data = self.graph[predecessor][concept]
            for edge in edge_data.values():
                relation = edge.get('relation', 'Unknown')
                weight = edge.get('weight', 1.0)
                results.append((predecessor, relation, concept, weight, 'incoming'))
                print(f"   {predecessor} --{relation}--> {concept} (weight: {weight:.2f})")
        
        return results
    
    def find_path(self, start_concept, end_concept, max_length=3):
        """
        Find connection paths between two concepts
        """
        if start_concept not in self.graph or end_concept not in self.graph:
            print(f"❓ One or both concepts not found in graph")
            return []
        
        try:
            # Find shortest path
            path = nx.shortest_path(self.graph, start_concept, end_concept, weight=None)
            
            print(f"🛤️  Path from '{start_concept}' to '{end_concept}':")
            
            # Print the path with relations
            for i in range(len(path) - 1):
                current = path[i]
                next_node = path[i + 1]
                
                if self.graph.has_edge(current, next_node):
                    edge_data = list(self.graph[current][next_node].values())[0]
                    relation = edge_data.get('relation', 'Unknown')
                    print(f"   {current} --{relation}--> {next_node}")
                
            return path
            
        except nx.NetworkXNoPath:
            print(f"❌ No path found between '{start_concept}' and '{end_concept}'")
            return []

In [ ]:
# Initialize agent
agent = KnowledgeGraphAgent(validate_on_add=True, verbose=False)

# Test with sample data (replace with your actual cleaned data)
sample_data = pd.DataFrame({
    'start_concept': ['dog', 'cat', 'dog', 'happy', 'sad'],
    'end_concept': ['animal', 'animal', 'cat', 'emotion', 'emotion'],
    'relation_type': ['IsA', 'IsA', 'RelatedTo', 'IsA', 'IsA'],
    'edge_weight': [1.0, 1.0, 0.8, 1.0, 1.0]
})

print("📝 Loading sample data...")
agent.bulk_load_triples(sample_data)

print("\n🔍 Testing queries...")
agent.query_concept('dog')

print("\n🛤️  Testing path finding...")
agent.find_path('dog', 'emotion')

#### Phase 3.A 
***
Testing agent interaction

In [ ]:
print("\n" + "="*60)
print("🎯 TESTING THE AGENT")
print("="*60)

# Test 1: Query specific concepts
print("\n🔍 Test 1: Querying concept relationships")
test_concepts = ['dog', 'cat', 'animal', 'happy', 'food']

for concept in test_concepts:
    if concept in [str(c).lower() for c in sample_data['start_concept'].unique()]:
        print(f"\n--- Relationships for '{concept}' ---")
        agent.query_concept(concept, max_results=5)
        break

# Test 2: Find paths between concepts
print("\n🛤️  Test 2: Finding concept paths")
# Try to find a path between two concepts
start_concepts = sample_data['start_concept'].value_counts().head(5).index.tolist()
end_concepts = sample_data['end_concept'].value_counts().head(5).index.tolist()

if len(start_concepts) > 0 and len(end_concepts) > 0:
    start_test = str(start_concepts[0])
    end_test = str(end_concepts[0])
    print(f"\nTrying to find path from '{start_test}' to '{end_test}':")
    agent.find_path(start_test, end_test)

# Test 3: Validation effectiveness
print("\n🔬 Test 3: Adding duplicate to test validation")
if len(sample_data) > 0:
    first_row = sample_data.iloc[0]
    print(f"Attempting to add duplicate: {first_row['start_concept']} --{first_row['relation_type']}--> {first_row['end_concept']}")
    
    success = agent.add_triple(
        start=str(first_row['start_concept']),
        relation=str(first_row['relation_type']),
        end=str(first_row['end_concept']),
        weight=float(first_row['edge_weight'])
    )
    
    print(f"Duplicate addition {'succeeded' if success else 'was rejected (as expected)'}")

# Save the processed data for future use
print(f"\n💾 Saving processed data...")
output_path = os.path.join(os.path.dirname(EN_PATH), 'conceptnet_en_processed_for_graph.parquet.gzip')
cleaned_english_triples.to_parquet(output_path, index=False, compression='gzip')
print(f"   Saved to: {output_path}")

print(f"\n🎉 Phase 2 Complete!")
print(f"   Graph loaded with {agent.validation_stats['successful_adds']:,} validated triples")
print(f"   Ready for agent interactions and queries!")

# Phase 4 
***
Testing if the agent successfully integrates the stratidfied seed

In [ ]:
print("🧠 Initializing Knowledge Graph Agent with stratified seed...")
agent = KnowledgeGraphAgent(validate_on_add=True, verbose=False)

# Load the stratified sample
print(f"📊 Loading {len(stratified_seed_data):,} stratified triples...")
agent.bulk_load_triples(stratified_seed_data)

print("\n" + "="*60)
print("🎯 TESTING AGENT WITH STRATIFIED SEED")
print("="*60)

# Test 1: Show relation diversity
print("\n📊 Relation diversity in loaded graph:")
relation_stats = {}
for _, _, data in agent.graph.edges(data=True):
    relation = data.get('relation', 'Unknown')
    if relation not in relation_stats:
        relation_stats[relation] = 0
    relation_stats[relation] += 1

# Show top relations
sorted_relations = sorted(relation_stats.items(), key=lambda x: x[1], reverse=True)
print("Top 10 relations in graph:")
for relation, count in sorted_relations[:10]:
    print(f"   {relation}: {count:,}")

# Test 2: Quality check - show high-weight concepts
print(f"\n⚖️  High-quality relationships (weight > 0.8):")
high_quality_count = 0
for start, end, data in agent.graph.edges(data=True):
    if data.get('weight', 0) > 0.8:
        relation = data.get('relation', 'Unknown')
        weight = data.get('weight', 0)
        print(f"   {start} --{relation}--> {end} (weight: {weight:.3f})")
        high_quality_count += 1
        if high_quality_count >= 10:  # Limit output
            break

print(f"Total high-quality relationships: {high_quality_count:,}")

# Test 3: Concept connectivity analysis
print(f"\n🕸️  Connectivity analysis:")
node_degrees = dict(agent.graph.degree())
top_connected = sorted(node_degrees.items(), key=lambda x: x[1], reverse=True)[:10]

print("Most connected concepts:")
for concept, degree in top_connected:
    print(f"   {concept}: {degree} connections")

# Test 4: Sample queries on well-connected concepts
print(f"\n🔍 Testing queries on top concepts:")
for concept, degree in top_connected[:3]:
    print(f"\n--- Relationships for '{concept}' (degree: {degree}) ---")
    agent.query_concept(concept, max_results=5)

print(f"\n🎉 Agent successfully initialized with high-quality stratified seed!")
print(f"   Ready for knowledge graph reasoning and expansion!")

# Phase 5 
***
Test if the Agent memory graph is now interactive

In [ ]:
class MVPKnowledgeAgent:
    """
    Minimum Viable Product - Interactive Knowledge Graph Agent
    Core features: Explore, Reason, Validate, Learn
    """
    
    def __init__(self, base_agent):
        self.graph = base_agent.graph
        self.base_agent = base_agent
        self.reasoning_cache = {}  # Cache for expensive operations
        self.learning_log = []     # Track what the agent learns
        
        print("🚀 MVP Knowledge Agent initialized!")
        print(f"   Base knowledge: {self.graph.number_of_nodes():,} concepts")
        print(f"   Relationships: {self.graph.number_of_edges():,} edges")
    
    def explore_concept(self, concept, depth=2, max_results=20):
        """
        Core MVP Feature 1: Deep concept exploration
        Shows not just direct relationships, but relationships of relationships
        """
        print(f"🔍 EXPLORING: '{concept}' (depth: {depth})")
        print("="*50)
        
        if concept not in self.graph:
            # Try fuzzy matching
            similar = self._find_similar_concepts(concept)
            if similar:
                print(f"❓ '{concept}' not found. Did you mean: {', '.join(similar[:3])}?")
                return None
            else:
                print(f"❌ '{concept}' not found in knowledge base")
                return None
        
        exploration_results = {
            'target_concept': concept,
            'direct_relations': [],
            'extended_relations': [],
            'concept_clusters': [],
            'reasoning_paths': []
        }
        
        # Level 1: Direct relationships
        print(f"\n📍 DIRECT RELATIONSHIPS:")
        direct_count = 0
        for neighbor in self.graph.neighbors(concept):
            if direct_count >= max_results // 2:
                break
                
            edge_data = list(self.graph[concept][neighbor].values())[0]
            relation = edge_data.get('relation', 'Unknown')
            weight = edge_data.get('weight', 1.0)
            
            exploration_results['direct_relations'].append({
                'from': concept,
                'relation': relation,
                'to': neighbor,
                'weight': weight,
                'type': 'outgoing'
            })
            
            print(f"   {concept} --{relation}--> {neighbor} (w: {weight:.2f})")
            direct_count += 1
        
        # Include incoming relationships
        for predecessor in self.graph.predecessors(concept):
            if direct_count >= max_results // 2:
                break
                
            edge_data = list(self.graph[predecessor][concept].values())[0]
            relation = edge_data.get('relation', 'Unknown')
            weight = edge_data.get('weight', 1.0)
            
            exploration_results['direct_relations'].append({
                'from': predecessor,
                'relation': relation,
                'to': concept,
                'weight': weight,
                'type': 'incoming'
            })
            
            print(f"   {predecessor} --{relation}--> {concept} (w: {weight:.2f})")
            direct_count += 1
        
        # Level 2: Extended exploration (relationships of relationships)
        if depth > 1:
            print(f"\n🔄 EXTENDED RELATIONSHIPS (depth 2):")
            extended_concepts = set()
            
            # Get neighbors of neighbors
            for neighbor in list(self.graph.neighbors(concept))[:5]:  # Limit to prevent explosion
                for second_neighbor in list(self.graph.neighbors(neighbor))[:3]:
                    if second_neighbor != concept and second_neighbor not in extended_concepts:
                        extended_concepts.add(second_neighbor)
                        
                        # Get the relation chain
                        edge1 = list(self.graph[concept][neighbor].values())[0]
                        edge2 = list(self.graph[neighbor][second_neighbor].values())[0]
                        
                        relation1 = edge1.get('relation', 'Unknown')
                        relation2 = edge2.get('relation', 'Unknown')
                        
                        exploration_results['extended_relations'].append({
                            'path': [concept, neighbor, second_neighbor],
                            'relations': [relation1, relation2],
                            'reasoning': f"{concept} --{relation1}--> {neighbor} --{relation2}--> {second_neighbor}"
                        })
                        
                        print(f"   {concept} --{relation1}--> {neighbor} --{relation2}--> {second_neighbor}")
                        
                        if len(extended_concepts) >= max_results // 4:
                            break
                if len(extended_concepts) >= max_results // 4:
                    break
        
        return exploration_results
    
    def reason_about_relationship(self, concept1, concept2, max_paths=3):
        """
        Core MVP Feature 2: Reasoning about why two concepts might be related
        """
        print(f"🧠 REASONING: Why might '{concept1}' and '{concept2}' be related?")
        print("="*60)
        
        if concept1 not in self.graph or concept2 not in self.graph:
            missing = [c for c in [concept1, concept2] if c not in self.graph]
            print(f"❌ Concepts not found: {missing}")
            return None
        
        reasoning_results = {
            'concept1': concept1,
            'concept2': concept2,
            'direct_connection': None,
            'reasoning_paths': [],
            'shared_concepts': [],
            'relationship_strength': 0.0
        }
        
        # Check for direct connection
        if self.graph.has_edge(concept1, concept2):
            edge_data = list(self.graph[concept1][concept2].values())[0]
            relation = edge_data.get('relation', 'Unknown')
            weight = edge_data.get('weight', 1.0)
            
            reasoning_results['direct_connection'] = {
                'relation': relation,
                'weight': weight
            }
            
            print(f"✅ DIRECT CONNECTION FOUND:")
            print(f"   {concept1} --{relation}--> {concept2} (weight: {weight:.3f})")
            reasoning_results['relationship_strength'] = weight
        
        # Find indirect reasoning paths
        try:
            paths = list(nx.all_simple_paths(self.graph, concept1, concept2, cutoff=3))[:max_paths]
            
            if paths:
                print(f"\n🛤️  REASONING PATHS FOUND ({len(paths)}):")
                
                for i, path in enumerate(paths, 1):
                    path_relations = []
                    path_weights = []
                    path_description = []
                    
                    for j in range(len(path) - 1):
                        current = path[j]
                        next_node = path[j + 1]
                        
                        if self.graph.has_edge(current, next_node):
                            edge_data = list(self.graph[current][next_node].values())[0]
                            relation = edge_data.get('relation', 'Unknown')
                            weight = edge_data.get('weight', 1.0)
                            
                            path_relations.append(relation)
                            path_weights.append(weight)
                            path_description.append(f"{current} --{relation}--> {next_node}")
                    
                    avg_weight = np.mean(path_weights) if path_weights else 0.0
                    
                    reasoning_results['reasoning_paths'].append({
                        'path': path,
                        'relations': path_relations,
                        'avg_weight': avg_weight,
                        'description': ' → '.join(path_description)
                    })
                    
                    print(f"\n   Path {i} (strength: {avg_weight:.3f}):")
                    for desc in path_description:
                        print(f"      {desc}")
                    
                    # Update relationship strength
                    if avg_weight > reasoning_results['relationship_strength']:
                        reasoning_results['relationship_strength'] = avg_weight
        
        except nx.NetworkXNoPath:
            print(f"❌ No reasoning paths found between '{concept1}' and '{concept2}'")
        
        # Find shared connections (concepts both are related to)
        concept1_neighbors = set(self.graph.neighbors(concept1)) | set(self.graph.predecessors(concept1))
        concept2_neighbors = set(self.graph.neighbors(concept2)) | set(self.graph.predecessors(concept2))
        shared = concept1_neighbors & concept2_neighbors
        
        if shared:
            print(f"\n🤝 SHARED CONNECTIONS ({len(shared)}):")
            for shared_concept in list(shared)[:5]:  # Limit output
                reasoning_results['shared_concepts'].append(shared_concept)
                print(f"   Both relate to: '{shared_concept}'")
        
        # Generate reasoning strength score
        strength_score = reasoning_results['relationship_strength']
        if strength_score > 0.8:
            strength_desc = "STRONG"
        elif strength_score > 0.5:
            strength_desc = "MODERATE"
        elif strength_score > 0.2:
            strength_desc = "WEAK"
        else:
            strength_desc = "MINIMAL"
        
        print(f"\n📊 RELATIONSHIP STRENGTH: {strength_desc} ({strength_score:.3f})")
        
        return reasoning_results
    
    def validate_new_knowledge(self, start_concept, relation, end_concept, confidence=1.0):
        """
        Core MVP Feature 3: Validate if new knowledge makes sense
        """
        print(f"🔬 VALIDATING: {start_concept} --{relation}--> {end_concept}")
        print("="*50)
        
        validation_result = {
            'proposed_triple': (start_concept, relation, end_concept),
            'confidence': confidence,
            'validation_score': 0.0,
            'supporting_evidence': [],
            'contradictions': [],
            'recommendation': 'REJECT'
        }
        
        # Check 1: Do the concepts exist in our knowledge base?
        concept_familiarity = 0
        if start_concept in self.graph:
            concept_familiarity += 0.5
            print(f"✅ Know about '{start_concept}'")
        else:
            print(f"❓ Unknown concept: '{start_concept}'")
        
        if end_concept in self.graph:
            concept_familiarity += 0.5
            print(f"✅ Know about '{end_concept}'")
        else:
            print(f"❓ Unknown concept: '{end_concept}'")
        
        # Check 2: Look for supporting evidence
        if start_concept in self.graph:
            start_relations = [data.get('relation') for _, _, data in self.graph.edges(start_concept, data=True)]
            if relation in start_relations:
                validation_result['supporting_evidence'].append(f"'{start_concept}' commonly uses '{relation}' relation")
                print(f"✅ '{start_concept}' commonly uses '{relation}' relation")
        
        # Check 3: Look for contradictions
        if self.graph.has_edge(start_concept, end_concept):
            existing_relations = [data.get('relation') for data in self.graph[start_concept][end_concept].values()]
            contradictory_relations = self.base_agent.contradiction_rules.get(relation, [])
            
            for existing_rel in existing_relations:
                if existing_rel in contradictory_relations:
                    validation_result['contradictions'].append(f"Contradicts existing '{existing_rel}' relation")
                    print(f"⚠️  Contradicts existing '{existing_rel}' relation")
        
        # Calculate validation score
        score = 0.0
        score += concept_familiarity * 0.3  # 30% for concept familiarity
        score += len(validation_result['supporting_evidence']) * 0.4  # 40% for supporting evidence
        score -= len(validation_result['contradictions']) * 0.5  # Penalty for contradictions
        score += confidence * 0.3  # 30% for stated confidence
        
        validation_result['validation_score'] = max(0.0, min(1.0, score))  # Clamp to [0,1]
        
        # Make recommendation
        if validation_result['validation_score'] > 0.7:
            validation_result['recommendation'] = 'ACCEPT'
            recommendation_color = "✅"
        elif validation_result['validation_score'] > 0.4:
            validation_result['recommendation'] = 'REVIEW'
            recommendation_color = "⚠️ "
        else:
            validation_result['recommendation'] = 'REJECT'
            recommendation_color = "❌"
        
        print(f"\n📊 VALIDATION SCORE: {validation_result['validation_score']:.3f}")
        print(f"{recommendation_color} RECOMMENDATION: {validation_result['recommendation']}")
        
        return validation_result
    
    def learn_new_triple(self, start_concept, relation, end_concept, confidence=1.0, force=False):
        """
        Core MVP Feature 4: Learn and integrate new knowledge
        """
        print(f"🧠 LEARNING: {start_concept} --{relation}--> {end_concept}")
        
        # First validate the knowledge
        validation = self.validate_new_knowledge(start_concept, relation, end_concept, confidence)
        
        should_learn = force or validation['recommendation'] in ['ACCEPT', 'REVIEW']
        
        if should_learn:
            # Add to graph
            success = self.base_agent.add_triple(start_concept, relation, end_concept, confidence)
            
            if success:
                # Log the learning event
                learning_event = {
                    'timestamp': pd.Timestamp.now(),
                    'triple': (start_concept, relation, end_concept),
                    'confidence': confidence,
                    'validation_score': validation['validation_score'],
                    'method': 'forced' if force else 'validated'
                }
                self.learning_log.append(learning_event)
                
                print(f"✅ LEARNED: Successfully integrated new knowledge")
                print(f"   Knowledge base now has {self.graph.number_of_edges():,} relationships")
                return True
            else:
                print(f"❌ FAILED: Could not integrate (duplicate or error)")
                return False
        else:
            print(f"❌ REJECTED: Validation score too low ({validation['validation_score']:.3f})")
            return False
    
    def _find_similar_concepts(self, concept, threshold=0.7):
        """Helper: Find concepts similar to the input (simple string matching)"""
        concept_lower = concept.lower()
        similar = []
        
        for node in self.graph.nodes():
            node_lower = str(node).lower()
            if concept_lower in node_lower or node_lower in concept_lower:
                similar.append(str(node))
            if len(similar) >= 5:
                break
        
        return similar
    
    def interactive_session(self):
        """
        Core MVP Feature 5: Interactive exploration session
        """
        print("🎮 Starting Interactive Knowledge Exploration Session!")
        print("Commands: explore <concept>, reason <concept1> <concept2>, validate <start> <relation> <end>, learn <start> <relation> <end>, quit")
        print("="*80)
        
        while True:
            try:
                user_input = input("\n🤖 mvp_agent> ").strip()
                
                if user_input.lower() in ['quit', 'exit', 'q']:
                    print("👋 Goodbye! Thanks for exploring knowledge with me!")
                    break
                
                parts = user_input.split()
                if not parts:
                    continue
                
                command = parts[0].lower()
                
                if command == 'explore' and len(parts) >= 2:
                    concept = ' '.join(parts[1:])
                    self.explore_concept(concept)
                
                elif command == 'reason' and len(parts) >= 3:
                    concept1 = parts[1]
                    concept2 = ' '.join(parts[2:])
                    self.reason_about_relationship(concept1, concept2)
                
                elif command == 'validate' and len(parts) >= 4:
                    start = parts[1]
                    relation = parts[2]
                    end = ' '.join(parts[3:])
                    self.validate_new_knowledge(start, relation, end)
                
                elif command == 'learn' and len(parts) >= 4:
                    start = parts[1]
                    relation = parts[2]
                    end = ' '.join(parts[3:])
                    self.learn_new_triple(start, relation, end)
                
                else:
                    print("❓ Unknown command. Try: explore <concept>, reason <concept1> <concept2>, validate/learn <start> <relation> <end>, quit")
            
            except KeyboardInterrupt:
                print("\n👋 Session interrupted. Goodbye!")
                break
            except Exception as e:
                print(f"❌ Error: {e}")

# Initialize the MVP Agent
print("🚀 Initializing MVP Knowledge Agent...")
mvp_agent = MVPKnowledgeAgent(agent)

print("\n" + "="*60)
print("🎯 MVP AGENT READY - TESTING CORE FEATURES")
print("="*60)

# Test the core features
print("\n1️⃣ Testing Concept Exploration:")
mvp_agent.explore_concept('dog', depth=2)

print("\n2️⃣ Testing Relationship Reasoning:")
mvp_agent.reason_about_relationship('dog', 'animal')

print("\n3️⃣ Testing Knowledge Validation:")
mvp_agent.validate_new_knowledge('dog', 'IsA', 'mammal', confidence=0.9)

print("\n4️⃣ Testing Learning:")
mvp_agent.learn_new_triple('dog', 'CapableOf', 'barking', confidence=0.95)

print(f"\n🎉 MVP Agent is ready! All core features working.")
print(f"📊 Current knowledge: {mvp_agent.graph.number_of_nodes():,} concepts, {mvp_agent.graph.number_of_edges():,} relationships")
print(f"🧠 Learning events: {len(mvp_agent.learning_log)}")

# Phase 6: Full learning loop

In [ ]:
class AdaptiveTrainingLoop:
    """
    Self-improving training loop that processes the entire dataset until convergence
    Uses adaptive batching, quality thresholds, and convergence detection
    """
    
    def __init__(self, mvp_agent, full_dataset, config=None):
        self.agent = mvp_agent
        self.full_dataset = full_dataset.copy()
        self.training_history = []
        self.convergence_metrics = deque(maxlen=10)  # Rolling window for convergence
        
        # Training configuration with speed and verbosity controls
        self.config = config or {
            # Core training parameters
            'initial_batch_size': 1000,
            'max_batch_size': 10000,
            'min_batch_size': 100,
            'quality_threshold_start': 0.3,
            'quality_threshold_end': 0.7,
            'convergence_patience': 3,
            'max_epochs': 20,
            'validation_sample_size': 500,
            'adaptive_threshold': True,
            'learning_rate_decay': 0.95,
            
            # Speed controls
            'test_mode': False,              # Fast testing mode
            'test_sample_size': 10000,       # Max triples in test mode
            'skip_intelligent_batching': False,  # Use simple batching for speed
            'fast_validation': False,        # Skip expensive validation steps
            'vectorized_processing': True,   # Use pandas vectorization where possible
            
            # Verbosity controls  
            'verbose_level': 1,              # 0=quiet, 1=normal, 2=detailed, 3=debug
            'print_frequency': 100,          # Print every N triples processed
            'batch_progress': True,          # Show batch-level progress
            'real_time_stats': True,         # Print stats during processing
            'checkpoint_frequency': 1000     # Print checkpoint every N triples
        }
        
        self.current_epoch = 0
        self.current_batch_size = self.config['initial_batch_size']
        self.current_quality_threshold = self.config['quality_threshold_start']
        self.processed_count = 0
        self.last_checkpoint_time = time.time()
        
        # Apply speed optimizations if in test mode
        if self.config['test_mode']:
            print("⚡ FAST TEST MODE ENABLED")
            if len(self.full_dataset) > self.config['test_sample_size']:
                self.full_dataset = self.full_dataset.sample(
                    n=self.config['test_sample_size'], 
                    random_state=42
                ).reset_index(drop=True)
                print(f"   📊 Dataset limited to {len(self.full_dataset):,} triples")
            
            # Speed up other parameters
            self.config['max_epochs'] = min(3, self.config['max_epochs'])
            self.config['convergence_patience'] = 2
            print(f"   🏃 Max epochs reduced to {self.config['max_epochs']}")
        
        self._print(f"🎯 Adaptive Training Loop Initialized!", level=1)
        self._print(f"   Dataset size: {len(self.full_dataset):,} triples", level=1)
        self._print(f"   Starting batch size: {self.current_batch_size:,}", level=1)
        self._print(f"   Quality threshold: {self.current_quality_threshold:.3f}", level=1)
        self._print(f"   Verbosity level: {self.config['verbose_level']}", level=2)
    
    def _print(self, message, level=1):
        """Controlled printing based on verbosity level"""
        if self.config['verbose_level'] >= level:
            print(message)
    
    def calculate_graph_quality_metrics(self):
        """Calculate comprehensive quality metrics for the current graph state"""
        graph = self.agent.graph
        
        metrics = {
            'total_nodes': graph.number_of_nodes(),
            'total_edges': graph.number_of_edges(),
            'density': nx.density(graph),
            'avg_degree': np.mean([d for n, d in graph.degree()]) if graph.number_of_nodes() > 0 else 0,
            'connected_components': nx.number_weakly_connected_components(graph),
            'avg_weight': 0,
            'high_quality_edges': 0,
            'relation_diversity': 0
        }
        
        # Weight-based metrics
        weights = [data.get('weight', 1.0) for _, _, data in graph.edges(data=True)]
        if weights:
            metrics['avg_weight'] = np.mean(weights)
            metrics['high_quality_edges'] = sum(1 for w in weights if w > 0.7)
        
        # Relation diversity
        relations = [data.get('relation', 'Unknown') for _, _, data in graph.edges(data=True)]
        metrics['relation_diversity'] = len(set(relations))
        
        # Graph connectivity score (higher is better)
        if metrics['total_nodes'] > 0:
            metrics['connectivity_score'] = (metrics['avg_degree'] * metrics['density'] * 
                                           (1 / max(1, metrics['connected_components'])))
        else:
            metrics['connectivity_score'] = 0
        
        return metrics
    
    def create_intelligent_batch(self, remaining_data, batch_size):
        """
        Create an intelligent batch prioritizing:
        1. High-weight triples
        2. Concepts already in the graph (for better connectivity)
        3. Diverse relation types
        4. Novel concepts (for expansion)
        """
        if len(remaining_data) <= batch_size:
            return remaining_data.copy(), pd.DataFrame()
        
        # Fast mode: skip intelligent batching for speed
        if self.config['skip_intelligent_batching']:
            self._print(f"   📊 Creating simple batch of {batch_size:,} (fast mode)", level=2)
            batch = remaining_data.head(batch_size)
            remaining = remaining_data.tail(len(remaining_data) - batch_size)
            return batch, remaining
        
        self._print(f"   📊 Creating intelligent batch of {batch_size:,} from {len(remaining_data):,} remaining...", level=2)
        
        # Score each triple for training value (vectorized for speed)
        if self.config['vectorized_processing']:
            scores = self._calculate_batch_scores_vectorized(remaining_data)
        else:
            scores = self._calculate_batch_scores_iterative(remaining_data)
        
        # Add scores and sort
        remaining_data = remaining_data.copy()
        remaining_data['training_score'] = scores
        remaining_data = remaining_data.sort_values('training_score', ascending=False)
        
        # Take top-scored triples for batch
        batch = remaining_data.head(batch_size).drop('training_score', axis=1)
        remaining = remaining_data.tail(len(remaining_data) - batch_size).drop('training_score', axis=1)
        
        return batch, remaining
    
    def _calculate_batch_scores_vectorized(self, data):
        """Vectorized batch scoring for speed"""
        existing_concepts = set(self.agent.graph.nodes())
        
        # Weight component (40%)
        scores = data['edge_weight'] * 0.4
        
        # Connectivity component (30%) - vectorized
        start_known = data['start_concept'].isin(existing_concepts)
        end_known = data['end_concept'].isin(existing_concepts)
        
        connectivity_scores = np.where(
            start_known & end_known, 0.3,  # Both known
            np.where(start_known | end_known, 0.2, 0.1)  # One known or none
        )
        scores += connectivity_scores
        
        # Relation diversity (20%) - simplified for speed
        common_relations = {'IsA', 'RelatedTo', 'PartOf', 'UsedFor', 'CapableOf'}
        relation_scores = data['relation_type'].isin(common_relations).astype(float) * 0.2
        scores += relation_scores
        
        # Simple validation prediction (10%)
        scores += 0.1  # Simplified
        
        return scores.values
    
    def _calculate_batch_scores_iterative(self, remaining_data):
        """Original iterative scoring (slower but more accurate)"""
        scores = []
        existing_concepts = set(self.agent.graph.nodes())
        
        for idx, row in remaining_data.iterrows():
            score = 0
            
            # Weight component (40%)
            score += row['edge_weight'] * 0.4
            
            # Connectivity component (30%)
            start_known = row['start_concept'] in existing_concepts
            end_known = row['end_concept'] in existing_concepts
            
            if start_known and end_known:
                score += 0.3
            elif start_known or end_known:
                score += 0.2
            else:
                score += 0.1
            
            # Relation diversity component (20%)
            current_relations = [data.get('relation') for _, _, data in self.agent.graph.edges(data=True)]
            relation_counts = pd.Series(current_relations).value_counts()
            current_relation = row['relation_type']
            
            if current_relation not in relation_counts.index:
                score += 0.2
            else:
                relation_freq = relation_counts[current_relation] / len(current_relations)
                score += 0.2 * (1 - relation_freq)
            
            # Validation prediction component (10%)
            common_relations = ['IsA', 'RelatedTo', 'PartOf', 'UsedFor', 'CapableOf']
            if current_relation in common_relations:
                score += 0.1
            
            scores.append(score)
        
        return scores
    
    def validate_batch_quality(self, batch_sample_size=100):
        """
        Validate a sample of the remaining data to estimate quality
        Used for adaptive threshold adjustment
        """
        if len(self.full_dataset) <= batch_sample_size:
            sample = self.full_dataset
        else:
            sample = self.full_dataset.sample(n=batch_sample_size, random_state=42)
        
        validation_scores = []
        
        for _, row in sample.iterrows():
            validation = self.agent.validate_new_knowledge(
                row['start_concept'], 
                row['relation_type'], 
                row['end_concept'], 
                row['edge_weight']
            )
            validation_scores.append(validation['validation_score'])
        
        return {
            'mean_quality': np.mean(validation_scores),
            'std_quality': np.std(validation_scores),
            'high_quality_ratio': sum(1 for s in validation_scores if s > 0.7) / len(validation_scores)
        }
    
    def process_batch(self, batch):
        """Process a batch of triples with detailed tracking and verbose output"""
        batch_stats = {
            'attempted': len(batch),
            'accepted': 0,
            'rejected_duplicate': 0,
            'rejected_validation': 0,
            'rejected_contradiction': 0,
            'avg_validation_score': 0,
            'processing_time': 0
        }
        
        validation_scores = []
        start_time = time.time()
        
        self._print(f"   🔄 Processing batch of {len(batch):,} triples...", level=1)
        
        # Choose progress bar or simple counter based on verbosity
        if self.config['batch_progress'] and self.config['verbose_level'] >= 2:
            iterator = tqdm(batch.iterrows(), total=len(batch), desc="Processing batch", leave=False)
        else:
            iterator = batch.iterrows()
        
        for i, (_, row) in enumerate(iterator):
            # Print real-time progress
            if (self.config['real_time_stats'] and 
                self.config['verbose_level'] >= 3 and 
                (i + 1) % self.config['print_frequency'] == 0):
                current_rate = batch_stats['accepted'] / max(1, i + 1)
                self._print(f"      Progress: {i+1:,}/{len(batch):,} "
                          f"(acceptance: {current_rate:.3f})", level=3)
            
            # Fast validation mode
            if self.config['fast_validation']:
                # Simple validation - just check weight and existing relations
                validation_score = row['edge_weight']
                if self.agent.graph.has_edge(row['start_concept'], row['end_concept']):
                    validation_score *= 0.5  # Penalize duplicates
            else:
                # Full validation
                validation = self.agent.validate_new_knowledge(
                    row['start_concept'], 
                    row['relation_type'], 
                    row['end_concept'], 
                    row['edge_weight']
                )
                validation_score = validation['validation_score']
            
            validation_scores.append(validation_score)
            
            # Decide whether to learn based on current quality threshold
            if validation_score >= self.current_quality_threshold:
                success = self.agent.learn_new_triple(
                    row['start_concept'], 
                    row['relation_type'], 
                    row['end_concept'], 
                    row['edge_weight'],
                    force=False
                )
                
                if success:
                    batch_stats['accepted'] += 1
                    self.processed_count += 1
                    
                    # Checkpoint reporting
                    if (self.config['real_time_stats'] and 
                        self.processed_count % self.config['checkpoint_frequency'] == 0):
                        elapsed = time.time() - self.last_checkpoint_time
                        rate = self.config['checkpoint_frequency'] / elapsed
                        total_edges = self.agent.graph.number_of_edges()
                        self._print(f"    🏁 CHECKPOINT: {self.processed_count:,} processed "
                                  f"(rate: {rate:.1f}/sec, total edges: {total_edges:,})", level=2)
                        self.last_checkpoint_time = time.time()
                else:
                    batch_stats['rejected_duplicate'] += 1
            else:
                batch_stats['rejected_validation'] += 1
        
        batch_stats['avg_validation_score'] = np.mean(validation_scores)
        batch_stats['processing_time'] = time.time() - start_time
        
        # Detailed batch summary
        acceptance_rate = batch_stats['accepted'] / max(1, batch_stats['attempted'])
        processing_rate = batch_stats['attempted'] / batch_stats['processing_time']
        
        self._print(f"      ✅ Batch complete: {batch_stats['accepted']}/{batch_stats['attempted']} accepted "
                  f"({acceptance_rate:.3f})", level=1)
        self._print(f"      ⚡ Processing rate: {processing_rate:.1f} triples/sec", level=2)
        self._print(f"      📊 Avg validation score: {batch_stats['avg_validation_score']:.3f}", level=2)
        
        return batch_stats
    
    def update_training_parameters(self, epoch_stats):
        """
        Adaptively update training parameters based on performance
        """
        self._print(f"   🎛️  Updating training parameters...", level=2)
        
        # Adaptive batch size
        acceptance_rate = epoch_stats['total_accepted'] / max(1, epoch_stats['total_attempted'])
        
        if acceptance_rate > 0.8:  # High acceptance, increase batch size
            self.current_batch_size = min(
                self.config['max_batch_size'],
                int(self.current_batch_size * 1.2)
            )
        elif acceptance_rate < 0.3:  # Low acceptance, decrease batch size for quality
            self.current_batch_size = max(
                self.config['min_batch_size'],
                int(self.current_batch_size * 0.8)
            )
        
        # Adaptive quality threshold
        if self.config['adaptive_threshold']:
            # Gradually increase threshold as graph improves
            progress = self.current_epoch / self.config['max_epochs']
            self.current_quality_threshold = (
                self.config['quality_threshold_start'] + 
                progress * (self.config['quality_threshold_end'] - self.config['quality_threshold_start'])
            )
        
        self._print(f"      Batch size: {self.current_batch_size:,}", level=1)
        self._print(f"      Quality threshold: {self.current_quality_threshold:.3f}", level=1)
    
    def check_convergence(self, epoch_stats):
        """
        Check if the training has converged (no more meaningful improvements)
        """
        # Add current metrics to rolling window
        improvement_score = epoch_stats['total_accepted'] / max(1, epoch_stats['total_attempted'])
        self.convergence_metrics.append(improvement_score)
        
        if len(self.convergence_metrics) < self.config['convergence_patience']:
            return False
        
        # Check if improvement has plateaued
        recent_improvements = list(self.convergence_metrics)
        trend = np.polyfit(range(len(recent_improvements)), recent_improvements, 1)[0]
        
        # Convergence criteria
        avg_recent_improvement = np.mean(recent_improvements)
        improvement_std = np.std(recent_improvements)
        
        # More aggressive convergence in test mode
        convergence_threshold = (0.02 if self.config['test_mode'] 
                               else self.config.get('convergence_threshold', 0.05))
        
        has_converged = (
            avg_recent_improvement < convergence_threshold or
            (trend <= 0 and improvement_std < 0.02)
        )
        
        if has_converged:
            self._print(f"   🎯 CONVERGENCE DETECTED:", level=1)
            self._print(f"      Recent improvement: {avg_recent_improvement:.4f}", level=1)
            self._print(f"      Trend: {trend:.6f}", level=2)
            self._print(f"      Stability: {improvement_std:.4f}", level=2)
        
        return has_converged
    
    def train_until_convergence(self):
        """
        Main training loop - processes entire dataset until convergence
        Now with configurable speed and verbosity
        """
        self._print("🚀 STARTING ADAPTIVE TRAINING LOOP", level=1)
        self._print("="*60, level=1)
        
        remaining_data = self.full_dataset.copy()
        training_start_time = time.time()
        
        # Pre-training summary
        self._print(f"📊 Training Configuration:", level=1)
        self._print(f"   Dataset size: {len(remaining_data):,} triples", level=1)
        self._print(f"   Test mode: {'ON' if self.config['test_mode'] else 'OFF'}", level=1)
        self._print(f"   Fast validation: {'ON' if self.config['fast_validation'] else 'OFF'}", level=1)
        self._print(f"   Intelligent batching: {'OFF' if self.config['skip_intelligent_batching'] else 'ON'}", level=1)
        self._print(f"   Max epochs: {self.config['max_epochs']}", level=1)
        
        while self.current_epoch < self.config['max_epochs']:
            self.current_epoch += 1
            epoch_start_time = time.time()
            
            self._print(f"\n📅 EPOCH {self.current_epoch}/{self.config['max_epochs']}", level=1)
            self._print("-" * 40, level=1)
            
            # Calculate pre-epoch metrics
            pre_metrics = self.calculate_graph_quality_metrics()
            self._print(f"   Graph state: {pre_metrics['total_nodes']:,} nodes, {pre_metrics['total_edges']:,} edges", level=1)
            self._print(f"   Connectivity score: {pre_metrics['connectivity_score']:.3f}", level=2)
            self._print(f"   Average weight: {pre_metrics['avg_weight']:.3f}", level=2)
            
            # Process data in batches
            epoch_stats = {
                'total_attempted': 0,
                'total_accepted': 0,
                'total_rejected': 0,
                'batches_processed': 0,
                'avg_validation_score': 0
            }
            
            validation_scores = []
            batch_times = []
            
            while len(remaining_data) > 0:
                batch_start_time = time.time()
                
                # Create intelligent batch
                batch, remaining_data = self.create_intelligent_batch(
                    remaining_data, 
                    self.current_batch_size
                )
                
                # Process batch
                batch_stats = self.process_batch(batch)
                
                # Track timing
                batch_time = time.time() - batch_start_time
                batch_times.append(batch_time)
                
                # Update epoch statistics
                epoch_stats['total_attempted'] += batch_stats['attempted']
                epoch_stats['total_accepted'] += batch_stats['accepted']
                epoch_stats['total_rejected'] += (batch_stats['rejected_duplicate'] + 
                                                 batch_stats['rejected_validation'] + 
                                                 batch_stats['rejected_contradiction'])
                epoch_stats['batches_processed'] += 1
                validation_scores.append(batch_stats['avg_validation_score'])
                
                # Batch summary
                acceptance_rate = batch_stats['accepted'] / max(1, batch_stats['attempted'])
                remaining_count = len(remaining_data)
                
                self._print(f"      Batch {epoch_stats['batches_processed']}: "
                          f"{batch_stats['accepted']}/{batch_stats['attempted']} accepted "
                          f"({acceptance_rate:.3f}), {remaining_count:,} remaining", level=1)
                
                # Detailed batch info
                self._print(f"         Validation score: {batch_stats['avg_validation_score']:.3f}, "
                          f"Time: {batch_time:.1f}s", level=2)
                
                # Early stopping if acceptance rate is very low
                if acceptance_rate < 0.01 and epoch_stats['batches_processed'] > 3:
                    self._print(f"      ⚠️  Early batch stopping - acceptance rate too low ({acceptance_rate:.4f})", level=1)
                    break
            
            # Calculate epoch-level metrics
            epoch_stats['avg_validation_score'] = np.mean(validation_scores) if validation_scores else 0
            post_metrics = self.calculate_graph_quality_metrics()
            epoch_duration = time.time() - epoch_start_time
            avg_batch_time = np.mean(batch_times) if batch_times else 0
            
            # Calculate improvements
            node_growth = post_metrics['total_nodes'] - pre_metrics['total_nodes']
            edge_growth = post_metrics['total_edges'] - pre_metrics['total_edges']
            
            epoch_summary = {
                'epoch': self.current_epoch,
                'duration': epoch_duration,
                'acceptance_rate': epoch_stats['total_accepted'] / max(1, epoch_stats['total_attempted']),
                'node_growth': node_growth,
                'edge_growth': edge_growth,
                'final_nodes': post_metrics['total_nodes'],
                'final_edges': post_metrics['total_edges'],
                'avg_validation_score': epoch_stats['avg_validation_score'],
                'connectivity_score': post_metrics['connectivity_score'],
                'batches_processed': epoch_stats['batches_processed'],
                'avg_batch_time': avg_batch_time
            }
            
            self.training_history.append(epoch_summary)
            
            # Detailed epoch summary
            self._print(f"\n   📊 EPOCH {self.current_epoch} SUMMARY:", level=1)
            self._print(f"      Duration: {epoch_duration:.1f}s ({epoch_stats['batches_processed']} batches)", level=1)
            self._print(f"      Acceptance rate: {epoch_summary['acceptance_rate']:.3f}", level=1)
            self._print(f"      Growth: +{node_growth:,} nodes, +{edge_growth:,} edges", level=1)
            self._print(f"      Total: {post_metrics['total_nodes']:,} nodes, {post_metrics['total_edges']:,} edges", level=1)
            
            # Performance metrics
            total_processed = epoch_stats['total_attempted']
            processing_rate = total_processed / epoch_duration if epoch_duration > 0 else 0
            self._print(f"      Processing rate: {processing_rate:.1f} triples/sec", level=2)
            self._print(f"      Avg validation score: {epoch_stats['avg_validation_score']:.3f}", level=2)
            self._print(f"      Connectivity improvement: {post_metrics['connectivity_score'] - pre_metrics['connectivity_score']:.3f}", level=2)
            
            # Quality metrics
            quality_ratio = post_metrics['high_quality_edges'] / max(1, post_metrics['total_edges'])
            self._print(f"      High-quality edge ratio: {quality_ratio:.3f}", level=2)
            self._print(f"      Relation diversity: {post_metrics['relation_diversity']}", level=2)
            
            # Update parameters for next epoch
            self.update_training_parameters(epoch_stats)
            
            # Check for convergence
            if self.check_convergence(epoch_stats):
                self._print(f"\n🎯 CONVERGENCE ACHIEVED after {self.current_epoch} epochs!", level=1)
                break
            
            # Early stopping for very poor performance
            if epoch_summary['acceptance_rate'] < 0.005 and self.current_epoch > 1:
                self._print(f"\n⚠️  EARLY STOPPING - acceptance rate too low ({epoch_summary['acceptance_rate']:.4f})", level=1)
                break
            
            # Reset remaining data for next epoch
            remaining_data = self.full_dataset.copy()
            
            # Memory cleanup suggestion for large datasets
            if len(self.full_dataset) > 50000 and self.current_epoch % 2 == 0:
                self._print("      💾 Consider running garbage collection for memory optimization", level=3)
        
        total_training_time = time.time() - training_start_time
        final_metrics = self.calculate_graph_quality_metrics()
        
        # Final summary
        self._print(f"\n🎉 TRAINING COMPLETE!", level=1)
        self._print("="*60, level=1)
        self._print(f"Total training time: {total_training_time/60:.1f} minutes", level=1)
        self._print(f"Epochs completed: {self.current_epoch}", level=1)
        self._print(f"Final graph: {final_metrics['total_nodes']:,} nodes, {final_metrics['total_edges']:,} edges", level=1)
        
        # Performance summary
        total_processed = sum(h['final_edges'] - (self.training_history[0]['final_edges'] if self.training_history else 0) 
                            for h in self.training_history[-1:])
        avg_processing_rate = total_processed / total_training_time if total_training_time > 0 else 0
        self._print(f"Average processing rate: {avg_processing_rate:.1f} edges/sec", level=1)
        
        # Quality summary
        final_quality_ratio = final_metrics['high_quality_edges'] / max(1, final_metrics['total_edges'])
        self._print(f"Final high-quality edge ratio: {final_quality_ratio:.3f}", level=1)
        self._print(f"Final connectivity score: {final_metrics['connectivity_score']:.3f}", level=1)
        self._print(f"Relation diversity: {final_metrics['relation_diversity']} types", level=1)
        
        # Configuration used
        self._print(f"\nConfiguration used:", level=2)
        self._print(f"   Test mode: {'YES' if self.config['test_mode'] else 'NO'}", level=2)
        self._print(f"   Fast validation: {'YES' if self.config['fast_validation'] else 'NO'}", level=2)
        self._print(f"   Intelligent batching: {'NO' if self.config['skip_intelligent_batching'] else 'YES'}", level=2)
        
        return self.training_history
    
    def plot_training_progress(self):
        """Visualize training progress"""
        if not self.training_history:
            print("No training history to plot")
            return
        
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        epochs = [h['epoch'] for h in self.training_history]
        
        # Plot 1: Acceptance rate over time
        acceptance_rates = [h['acceptance_rate'] for h in self.training_history]
        axes[0,0].plot(epochs, acceptance_rates, 'b-o')
        axes[0,0].set_title('Acceptance Rate Over Time')
        axes[0,0].set_xlabel('Epoch')
        axes[0,0].set_ylabel('Acceptance Rate')
        axes[0,0].grid(True)
        
        # Plot 2: Graph growth
        total_edges = [h['final_edges'] for h in self.training_history]
        axes[0,1].plot(epochs, total_edges, 'g-o')
        axes[0,1].set_title('Knowledge Graph Growth')
        axes[0,1].set_xlabel('Epoch')
        axes[0,1].set_ylabel('Total Edges')
        axes[0,1].grid(True)
        
        # Plot 3: Validation scores
        val_scores = [h['avg_validation_score'] for h in self.training_history]
        axes[1,0].plot(epochs, val_scores, 'r-o')
        axes[1,0].set_title('Average Validation Score')
        axes[1,0].set_xlabel('Epoch')
        axes[1,0].set_ylabel('Validation Score')
        axes[1,0].grid(True)
        
        # Plot 4: Edge growth per epoch
        edge_growth = [h['edge_growth'] for h in self.training_history]
        axes[1,1].bar(epochs, edge_growth, alpha=0.7)
        axes[1,1].set_title('New Edges Per Epoch')
        axes[1,1].set_xlabel('Epoch')
        axes[1,1].set_ylabel('New Edges Added')
        axes[1,1].grid(True)
        
        plt.tight_layout()
        plt.show()


In [34]:
# Usage example - Updated with speed and verbosity controls
print("🎯 Setting up Configurable Training Loop...")

# FAST TEST CONFIGURATION - for development and testing
fast_test_config = {
    # Speed optimizations
    'test_mode': True,                   # Enable fast mode
    'test_sample_size': 5000,           # Only 5K triples for quick testing
    'initial_batch_size': 200,          # Smaller batches
    'max_batch_size': 1000,            # Lower max batch size
    'min_batch_size': 50,
    'skip_intelligent_batching': True,  # Simple batching for speed
    'fast_validation': True,            # Skip expensive validation
    'vectorized_processing': True,      # Use pandas vectorization
    
    # Training parameters
    'quality_threshold_start': 0.4,     # More permissive
    'quality_threshold_end': 0.6,       # Still maintain some quality
    'max_epochs': 3,                    # Just 3 epochs for testing
    'convergence_patience': 2,          # Quick convergence detection
    'adaptive_threshold': False,        # Fixed threshold for simplicity
    
    # Verbosity controls
    'verbose_level': 3,                 # Maximum verbosity
    'print_frequency': 25,              # Print every 25 triples
    'batch_progress': True,             # Show progress bars
    'real_time_stats': True,            # Real-time statistics
    'checkpoint_frequency': 200         # Checkpoint every 200 triples
}
# PRODUCTION CONFIGURATION - for full training runs
production_config = {
   # Full processing
   'test_mode': False,
   'skip_intelligent_batching': False,
   'fast_validation': False,
   'vectorized_processing': True,
   
   # Standard training parameters
   'initial_batch_size': 2000,
   'max_batch_size': 10000,
   'min_batch_size': 100,
   'quality_threshold_start': 0.3,
   'quality_threshold_end': 0.7,
   'max_epochs': 15,
   'convergence_patience': 3,
   'adaptive_threshold': True,
   
   # Moderate verbosity
   'verbose_level': 2,
   'print_frequency': 100,
   'batch_progress': True,
   'real_time_stats': True,
   'checkpoint_frequency': 1000
}

# SILENT FAST MODE - for background processing
silent_fast_config = {
   'test_mode': True,
   'test_sample_size': 10000,
   'initial_batch_size': 500,
   'max_batch_size': 2000,
   'min_batch_size': 100,
   'skip_intelligent_batching': True,
   'fast_validation': True,
   'max_epochs': 5,
   'quality_threshold_start': 0.4,
   'quality_threshold_end': 0.6,
   'convergence_patience': 2,
   'adaptive_threshold': False,
   
   # Minimal verbosity
   'verbose_level': 1,
   'print_frequency': 1000,
   'batch_progress': False,
   'real_time_stats': False,
   'checkpoint_frequency': 2000
}

🎯 Setting up Configurable Training Loop...


In [35]:
# Choose your configuration
print("📋 Available configurations:")
print("   1. fast_test_config - Quick testing with maximum verbosity")
print("   2. production_config - Full training with moderate verbosity") 
print("   3. silent_fast_config - Fast training with minimal output")

📋 Available configurations:
   1. fast_test_config - Quick testing with maximum verbosity
   2. production_config - Full training with moderate verbosity
   3. silent_fast_config - Fast training with minimal output


In [36]:
# Initialize with fast test config (change as needed)
trainer = AdaptiveTrainingLoop(mvp_agent, cleaned_english_triples, fast_test_config)

print(f"\n🚀 Ready to train with FAST TEST configuration!")
print(f"⚡ This should take ~2-5 minutes instead of hours")
print(f"🔊 Maximum verbosity - you'll see everything happening")
print(f"🎯 Start training with:")
print(f"   training_history = trainer.train_until_convergence()")
training_history = trainer.train_until_convergence()

⚡ FAST TEST MODE ENABLED
   📊 Dataset limited to 5,000 triples
   🏃 Max epochs reduced to 3
🎯 Adaptive Training Loop Initialized!
   Dataset size: 5,000 triples
   Starting batch size: 200
   Quality threshold: 0.400
   Verbosity level: 3

🚀 Ready to train with FAST TEST configuration!
⚡ This should take ~2-5 minutes instead of hours
🔊 Maximum verbosity - you'll see everything happening
🎯 Start training with:
   training_history = trainer.train_until_convergence()
🚀 STARTING ADAPTIVE TRAINING LOOP
📊 Training Configuration:
   Dataset size: 5,000 triples
   Test mode: ON
   Fast validation: ON
   Intelligent batching: OFF
   Max epochs: 3

📅 EPOCH 1/3
----------------------------------------
   Graph state: 7,542 nodes, 7,978 edges
   Connectivity score: 0.000
   Average weight: 2.907
   📊 Creating simple batch of 200 (fast mode)
   🔄 Processing batch of 200 triples...


🧠 LEARNING: v --FormOf--> swinck
🔬 VALIDATING: v --FormOf--> swinck
✅ Know about 'v'
✅ Know about 'swinck'
✅ 'v' commonly uses 'FormOf' relation

📊 VALIDATION SCORE: 1.000
✅ RECOMMENDATION: ACCEPT
❌ FAILED: Could not integrate (duplicate or error)
🧠 LEARNING: superviolent --DerivedFrom--> violent
🔬 VALIDATING: superviolent --DerivedFrom--> violent
❓ Unknown concept: 'superviolent'
❓ Unknown concept: 'violent'

📊 VALIDATION SCORE: 0.300
❌ RECOMMENDATION: REJECT
❌ REJECTED: Validation score too low (0.300)
🧠 LEARNING: purple_sand_tilefish --IsA--> n
🔬 VALIDATING: purple_sand_tilefish --IsA--> n
✅ Know about 'purple_sand_tilefish'
✅ Know about 'n'
✅ 'purple_sand_tilefish' commonly uses 'IsA' relation

📊 VALIDATION SCORE: 1.000
✅ RECOMMENDATION: ACCEPT
❌ FAILED: Could not integrate (duplicate or error)
🧠 LEARNING: n --FormOf--> febricity
🔬 VALIDATING: n --FormOf--> febricity
✅ Know about 'n'
✅ Know about 'febricity'
✅ 'n' commonly uses 'FormOf' relation

📊 VALIDATION SCORE: 1.000
✅ RECOMME

      ✅ Batch complete: 0/200 accepted (0.000)
      ⚡ Processing rate: 2366.6 triples/sec
      📊 Avg validation score: 0.606
      Batch 1: 0/200 accepted (0.000), 4,800 remaining
         Validation score: 0.606, Time: 0.1s
   📊 Creating simple batch of 200 (fast mode)
   🔄 Processing batch of 200 triples...


🧠 LEARNING: a --RelatedTo--> christofascism
🔬 VALIDATING: a --RelatedTo--> christofascism
✅ Know about 'a'
✅ Know about 'christofascism'
✅ 'a' commonly uses 'RelatedTo' relation

📊 VALIDATION SCORE: 1.000
✅ RECOMMENDATION: ACCEPT
❌ FAILED: Could not integrate (duplicate or error)
🧠 LEARNING: n --RelatedTo--> gluconic_acid
🔬 VALIDATING: n --RelatedTo--> gluconic_acid
✅ Know about 'n'
✅ Know about 'gluconic_acid'
✅ 'n' commonly uses 'RelatedTo' relation

📊 VALIDATION SCORE: 1.000
✅ RECOMMENDATION: ACCEPT
❌ FAILED: Could not integrate (duplicate or error)
🧠 LEARNING: n --FormOf--> lat
🔬 VALIDATING: n --FormOf--> lat
✅ Know about 'n'
✅ Know about 'lat'
✅ 'n' commonly uses 'FormOf' relation

📊 VALIDATION SCORE: 1.000
✅ RECOMMENDATION: ACCEPT
❌ FAILED: Could not integrate (duplicate or error)
🧠 LEARNING: mycobiome --DerivedFrom--> n
🔬 VALIDATING: mycobiome --DerivedFrom--> n
✅ Know about 'mycobiome'
✅ Know about 'n'
✅ 'mycobiome' commonly uses 'DerivedFrom' relation

📊 VALIDATION SCORE: 1.00

Processing batch:   0%|          | 0/200 [00:00<?, ?it/s]

...

Processing batch:   0%|          | 0/160 [00:00<?, ?it/s]

🧠 LEARNING: reception_area --AtLocation--> office_building
🔬 VALIDATING: reception_area --AtLocation--> office_building
❓ Unknown concept: 'reception_area'
❓ Unknown concept: 'office_building'

📊 VALIDATION SCORE: 0.300
❌ RECOMMENDATION: REJECT
❌ REJECTED: Validation score too low (0.300)
🧠 LEARNING: v --FormOf--> play_both_sides_against_middle
🔬 VALIDATING: v --FormOf--> play_both_sides_against_middle
✅ Know about 'v'
✅ Know about 'play_both_sides_against_middle'
✅ 'v' commonly uses 'FormOf' relation

📊 VALIDATION SCORE: 1.000
✅ RECOMMENDATION: ACCEPT
❌ FAILED: Could not integrate (duplicate or error)
🧠 LEARNING: a --RelatedTo--> adrenalized
🔬 VALIDATING: a --RelatedTo--> adrenalized
✅ Know about 'a'
✅ Know about 'adrenalized'
✅ 'a' commonly uses 'RelatedTo' relation

📊 VALIDATION SCORE: 1.000
✅ RECOMMENDATION: ACCEPT
❌ FAILED: Could not integrate (duplicate or error)
🧠 LEARNING: a --RelatedTo--> onshore
🔬 VALIDATING: a --RelatedTo--> onshore
✅ Know about 'a'
✅ Know about 'onshore'
✅ 


📊 VALIDATION SCORE: 1.000
✅ RECOMMENDATION: ACCEPT
❌ FAILED: Could not integrate (duplicate or error)
🧠 LEARNING: carbon --IsA--> element
🔬 VALIDATING: carbon --IsA--> element
✅ Know about 'carbon'
✅ Know about 'element'
✅ 'carbon' commonly uses 'IsA' relation

📊 VALIDATION SCORE: 1.000
✅ RECOMMENDATION: ACCEPT
❌ FAILED: Could not integrate (duplicate or error)
🧠 LEARNING: n --FormOf--> liber
🔬 VALIDATING: n --FormOf--> liber
✅ Know about 'n'
✅ Know about 'liber'
✅ 'n' commonly uses 'FormOf' relation

📊 VALIDATION SCORE: 1.000
✅ RECOMMENDATION: ACCEPT
❌ FAILED: Could not integrate (duplicate or error)
🧠 LEARNING: southern_pochard --IsA--> n
🔬 VALIDATING: southern_pochard --IsA--> n
❓ Unknown concept: 'southern_pochard'
✅ Know about 'n'

📊 VALIDATION SCORE: 0.300
❌ RECOMMENDATION: REJECT
❌ REJECTED: Validation score too low (0.300)
🧠 LEARNING: n --RelatedTo--> unking
🔬 VALIDATING: n --RelatedTo--> unking
✅ Know about 'n'
✅ Know about 'unking'
✅ 'n' commonly uses 'RelatedTo' relation

📊

      ✅ Batch complete: 0/160 accepted (0.000)
      ⚡ Processing rate: 1818.2 triples/sec
      📊 Avg validation score: 0.603
      Batch 3: 0/160 accepted (0.000), 4,520 remaining
         Validation score: 0.603, Time: 0.1s
   📊 Creating simple batch of 160 (fast mode)
   🔄 Processing batch of 160 triples...


🧠 LEARNING: nonbirational --DerivedFrom--> birational
🔬 VALIDATING: nonbirational --DerivedFrom--> birational
❓ Unknown concept: 'nonbirational'
❓ Unknown concept: 'birational'

📊 VALIDATION SCORE: 0.300
❌ RECOMMENDATION: REJECT
❌ REJECTED: Validation score too low (0.300)
🧠 LEARNING: squintest --DerivedFrom--> squint
🔬 VALIDATING: squintest --DerivedFrom--> squint
❓ Unknown concept: 'squintest'
❓ Unknown concept: 'squint'

📊 VALIDATION SCORE: 0.300
❌ RECOMMENDATION: REJECT
❌ REJECTED: Validation score too low (0.300)
🧠 LEARNING: n --FormOf--> breviary
🔬 VALIDATING: n --FormOf--> breviary
✅ Know about 'n'
✅ Know about 'breviary'
✅ 'n' commonly uses 'FormOf' relation

📊 VALIDATION SCORE: 1.000
✅ RECOMMENDATION: ACCEPT
❌ FAILED: Could not integrate (duplicate or error)
🧠 LEARNING: a --RelatedTo--> unprejudiced
🔬 VALIDATING: a --RelatedTo--> unprejudiced
✅ Know about 'a'
✅ Know about 'unprejudiced'
✅ 'a' commonly uses 'RelatedTo' relation

📊 VALIDATION SCORE: 1.000
✅ RECOMMENDATION: ACCEP

In [39]:
# Quick training starter for immediate testing
print(f"\n🏃 QUICK START - uncomment to begin:")
print(f"# training_history = trainer.train_until_convergence()")
print(f"# trainer.plot_training_progress()")  # Show results when done
# Test with 50K triples first
trainer_50k = AdaptiveTrainingLoop(mvp_agent, cleaned_english_triples.head(50000), silent_fast_config)
training_history = trainer_50k.train_until_convergence()


🏃 QUICK START - uncomment to begin:
# training_history = trainer.train_until_convergence()
# trainer.plot_training_progress()
⚡ FAST TEST MODE ENABLED
   📊 Dataset limited to 10,000 triples
   🏃 Max epochs reduced to 3
🎯 Adaptive Training Loop Initialized!
   Dataset size: 10,000 triples
   Starting batch size: 500
   Quality threshold: 0.400
🚀 STARTING ADAPTIVE TRAINING LOOP
📊 Training Configuration:
   Dataset size: 10,000 triples
   Test mode: ON
   Fast validation: ON
   Intelligent batching: OFF
   Max epochs: 3

📅 EPOCH 1/3
----------------------------------------
   Graph state: 7,542 nodes, 7,978 edges
   🔄 Processing batch of 500 triples...
🧠 LEARNING: moldy_food --AtLocation--> trash
🔬 VALIDATING: moldy_food --AtLocation--> trash
❓ Unknown concept: 'moldy_food'
✅ Know about 'trash'

📊 VALIDATION SCORE: 0.450
⚠️  RECOMMENDATION: REVIEW
✅ LEARNED: Successfully integrated new knowledge
   Knowledge base now has 7,979 relationships
🧠 LEARNING: n --Antonym--> passifan
🔬 VALIDATIN

KeyboardInterrupt: 

In [38]:
print(f"\n🚀 Ready to train on {len(cleaned_english_triples):,} triples!")
print(f"⚡ This will take several hours but creates a self-improving agent")
print(f"🎯 Uncomment the next line to start training:")
print(f"# training_history = trainer.train_until_convergence()")
trainer = AdaptiveTrainingLoop(mvp_agent, cleaned_english_triples, production_config)


🚀 Ready to train on 1,655,522 triples!
⚡ This will take several hours but creates a self-improving agent
🎯 Uncomment the next line to start training:
# training_history = trainer.train_until_convergence()
🎯 Adaptive Training Loop Initialized!
   Dataset size: 1,655,522 triples
   Starting batch size: 2,000
   Quality threshold: 0.300
   Verbosity level: 2


# Phase 7
***
Testing suite for current trained knowledge graph agent

In [ ]:
# Knowledge Graph Query Testing Suite
# Test what your agent learned during training

class QueryTestingSuite:
    """
    Comprehensive testing suite for your trained knowledge graph agent
    """
    
    def __init__(self, mvp_agent):
        self.agent = mvp_agent
        self.graph = mvp_agent.graph
        
        print("🔍 Query Testing Suite Initialized!")
        print(f"   Knowledge base: {self.graph.number_of_nodes():,} concepts")
        print(f"   Relationships: {self.graph.number_of_edges():,} edges")
        print(f"   Learning events: {len(mvp_agent.learning_log) if hasattr(mvp_agent, 'learning_log') else 'N/A'}")
    
    def analyze_knowledge_structure(self):
        """Get an overview of what the agent learned"""
        print("\n📊 KNOWLEDGE STRUCTURE ANALYSIS")
        print("="*50)
        
        # Most connected concepts
        node_degrees = dict(self.graph.degree())
        top_connected = sorted(node_degrees.items(), key=lambda x: x[1], reverse=True)[:15]
        
        print("🔗 Most Connected Concepts:")
        for concept, degree in top_connected:
            print(f"   {concept}: {degree} connections")
        
        # Relation type distribution
        relation_counts = Counter()
        for _, _, data in self.graph.edges(data=True):
            relation_counts[data.get('relation', 'Unknown')] += 1
        
        print(f"\n🏷️  Relation Types Learned ({len(relation_counts)} total):")
        for relation, count in relation_counts.most_common(10):
            print(f"   {relation}: {count:,}")
        
        # Quality distribution
        weights = [data.get('weight', 1.0) for _, _, data in self.graph.edges(data=True)]
        weight_stats = pd.Series(weights).describe()
        
        print(f"\n⚖️  Relationship Quality:")
        print(f"   Average weight: {weight_stats['mean']:.3f}")
        print(f"   High quality (>0.8): {sum(1 for w in weights if w > 0.8):,}")
        print(f"   Medium quality (0.5-0.8): {sum(1 for w in weights if 0.5 <= w <= 0.8):,}")
        print(f"   Low quality (<0.5): {sum(1 for w in weights if w < 0.5):,}")
        
        return {
            'top_concepts': top_connected,
            'relations': relation_counts,
            'quality_stats': weight_stats
        }
    
    def test_basic_queries(self):
        """Test basic concept exploration queries"""
        print("\n🔍 BASIC QUERY TESTING")
        print("="*50)
        
        # Get some well-connected concepts to test
        node_degrees = dict(self.graph.degree())
        test_concepts = [concept for concept, degree in 
                        sorted(node_degrees.items(), key=lambda x: x[1], reverse=True)[:10]
                        if degree > 5]  # Only test concepts with decent connections
        
        if not test_concepts:
            print("❌ No well-connected concepts found for testing")
            return
        
        print(f"Testing with concepts: {test_concepts[:5]}")
        
        for concept in test_concepts[:3]:  # Test first 3
            print(f"\n--- Testing '{concept}' ---")
            try:
                results = self.agent.explore_concept(concept, depth=1, max_results=8)
                if results and 'direct_relations' in results:
                    print(f"✅ Found {len(results['direct_relations'])} direct relationships")
                else:
                    print("❌ No relationships found")
            except Exception as e:
                print(f"❌ Error exploring '{concept}': {e}")
    
    def test_reasoning_queries(self):
        """Test relationship reasoning between concepts"""
        print("\n🧠 REASONING QUERY TESTING")
        print("="*50)
        
        # Find pairs of concepts that might be interestingly related
        test_pairs = self._find_interesting_concept_pairs()
        
        for concept1, concept2 in test_pairs[:3]:
            print(f"\n--- Reasoning: '{concept1}' ↔ '{concept2}' ---")
            try:
                results = self.agent.reason_about_relationship(concept1, concept2)
                if results and results.get('relationship_strength', 0) > 0:
                    strength = results['relationship_strength']
                    print(f"✅ Relationship strength: {strength:.3f}")
                else:
                    print("❌ No meaningful relationship found")
            except Exception as e:
                print(f"❌ Error reasoning about '{concept1}' and '{concept2}': {e}")
    
    def test_validation_system(self):
        """Test the knowledge validation system with new triples"""
        print("\n🔬 VALIDATION SYSTEM TESTING")
        print("="*50)
        
        # Test cases: [start, relation, end, expected_outcome]
        test_cases = [
            # Should be accepted (logical)
            ("dog", "IsA", "animal", "ACCEPT"),
            ("car", "UsedFor", "transportation", "ACCEPT"),
            ("book", "UsedFor", "reading", "ACCEPT"),
            
            # Should be rejected (illogical)
            ("rock", "CapableOf", "singing", "REJECT"),
            ("number", "AtLocation", "refrigerator", "REJECT"),
            
            # Might be reviewed (uncertain)
            ("computer", "RelatedTo", "thinking", "REVIEW"),
            ("music", "Causes", "happiness", "REVIEW")
        ]
        
        print("Testing validation on new knowledge:")
        
        for start, relation, end, expected in test_cases:
            print(f"\n   Testing: {start} --{relation}--> {end}")
            try:
                validation = self.agent.validate_new_knowledge(start, relation, end, confidence=0.8)
                actual = validation['recommendation']
                score = validation['validation_score']
                
                status = "✅" if actual == expected else "⚠️"
                print(f"   {status} Expected: {expected}, Got: {actual} (score: {score:.3f})")
                
            except Exception as e:
                print(f"   ❌ Error validating: {e}")
    
    def test_learned_knowledge(self):
        """Test specific knowledge that should have been learned"""
        print("\n📚 LEARNED KNOWLEDGE TESTING")
        print("="*50)
        
        # Test some relationships we saw being learned in the output
        learned_examples = [
            ("door", "bus"),  # door --AtLocation--> bus
            ("fruit", "sugar"),  # fruit --HasA--> sugar  
            ("hand", "grabbing"),  # hand --RelatedTo--> grabbing
            ("her", "pronoun"),  # her --RelatedTo--> pronoun
            ("money", "dough"),  # money --RelatedTo--> dough
        ]
        
        print("Checking specific learned relationships:")
        
        for concept1, concept2 in learned_examples:
            print(f"\n   Checking connection: {concept1} ↔ {concept2}")
            
            # Check if both concepts exist
            if concept1 not in self.graph or concept2 not in self.graph:
                missing = [c for c in [concept1, concept2] if c not in self.graph]
                print(f"   ❌ Missing concepts: {missing}")
                continue
            
            # Check for direct connection
            connected = False
            relation_found = None
            
            if self.graph.has_edge(concept1, concept2):
                edge_data = list(self.graph[concept1][concept2].values())[0]
                relation_found = edge_data.get('relation', 'Unknown')
                connected = True
            elif self.graph.has_edge(concept2, concept1):
                edge_data = list(self.graph[concept2][concept1].values())[0]
                relation_found = edge_data.get('relation', 'Unknown')
                connected = True
            
            if connected:
                print(f"   ✅ Connected via: {relation_found}")
            else:
                # Check for indirect connection
                try:
                    path = self.agent.reason_about_relationship(concept1, concept2)
                    if path and path.get('reasoning_paths'):
                        print(f"   🔄 Indirectly connected via reasoning")
                    else:
                        print(f"   ❌ No connection found")
                except:
                    print(f"   ❌ No connection found")
    
    def interactive_query_session(self):
        """Start an interactive query session"""
        print("\n🎮 INTERACTIVE QUERY SESSION")
        print("="*50)
        print("Commands:")
        print("  explore <concept> - Explore a concept's relationships")
        print("  reason <concept1> <concept2> - Find relationships between concepts")
        print("  validate <start> <relation> <end> - Test knowledge validation")
        print("  random - Get random concepts to explore")
        print("  stats - Show knowledge statistics")
        print("  quit - Exit session")
        
        while True:
            try:
                user_input = input("\n🤖 query> ").strip()
                
                if user_input.lower() in ['quit', 'exit', 'q']:
                    print("👋 Query session ended!")
                    break
                
                parts = user_input.split()
                if not parts:
                    continue
                
                command = parts[0].lower()
                
                if command == 'explore' and len(parts) >= 2:
                    concept = ' '.join(parts[1:])
                    self.agent.explore_concept(concept, depth=2, max_results=10)
                
                elif command == 'reason' and len(parts) >= 3:
                    concept1 = parts[1]
                    concept2 = ' '.join(parts[2:])
                    self.agent.reason_about_relationship(concept1, concept2)
                
                elif command == 'validate' and len(parts) >= 4:
                    start = parts[1]
                    relation = parts[2]
                    end = ' '.join(parts[3:])
                    result = self.agent.validate_new_knowledge(start, relation, end)
                    print(f"Validation: {result['recommendation']} (score: {result['validation_score']:.3f})")
                
                elif command == 'random':
                    concepts = list(self.graph.nodes())
                    random_concepts = random.sample(concepts, min(10, len(concepts)))
                    print(f"Random concepts to explore: {random_concepts}")
                
                elif command == 'stats':
                    print(f"Concepts: {self.graph.number_of_nodes():,}")
                    print(f"Relationships: {self.graph.number_of_edges():,}")
                    node_degrees = dict(self.graph.degree())
                    top_concept = max(node_degrees.items(), key=lambda x: x[1])
                    print(f"Most connected: {top_concept[0]} ({top_concept[1]} connections)")
                
                else:
                    print("❓ Unknown command. Try: explore <concept>, reason <concept1> <concept2>, validate <start> <relation> <end>, random, stats, quit")
            
            except KeyboardInterrupt:
                print("\n👋 Query session interrupted!")
                break
            except Exception as e:
                print(f"❌ Error: {e}")
    
    def _find_interesting_concept_pairs(self):
        """Find pairs of concepts that might have interesting relationships"""
        # Get some well-connected concepts
        node_degrees = dict(self.graph.degree())
        top_concepts = [concept for concept, degree in 
                       sorted(node_degrees.items(), key=lambda x: x[1], reverse=True)[:20]]
        
        # Create some potentially interesting pairs
        pairs = []
        for i in range(min(5, len(top_concepts))):
            for j in range(i+1, min(i+4, len(top_concepts))):
                pairs.append((top_concepts[i], top_concepts[j]))
        
        return pairs[:10]  # Return up to 10 pairs
    
    def run_full_test_suite(self):
        """Run all tests in sequence"""
        print("🚀 RUNNING FULL QUERY TEST SUITE")
        print("="*60)
        
        structure_info = self.analyze_knowledge_structure()
        self.test_basic_queries()
        self.test_reasoning_queries()
        self.test_validation_system()
        self.test_learned_knowledge()
        
        print("\n🎯 TEST SUITE COMPLETE!")
        print("="*60)
        print("💡 Try the interactive session for custom queries:")
        print("   tester.interactive_query_session()")
        
        return structure_info

# Initialize the testing suite
print("🔍 Initializing Query Testing Suite...")
tester = QueryTestingSuite(mvp_agent)

print("\n📋 Available Testing Options:")
print("   1. tester.run_full_test_suite() - Run all tests")
print("   2. tester.analyze_knowledge_structure() - See what was learned")
print("   3. tester.test_basic_queries() - Test concept exploration")
print("   4. tester.test_reasoning_queries() - Test relationship reasoning")
print("   5. tester.test_learned_knowledge() - Check specific learned facts")
print("   6. tester.interactive_query_session() - Interactive exploration")

print(f"\n🎯 Quick start - run the full test suite:")
print(f"   results = tester.run_full_test_suite()")

In [ ]:
# Run the full test suite to see everything your agent learned
results = tester.run_full_test_suite()